In [1]:
from transformers import pipeline

/Users/john/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text_classification_pipeline = pipeline(
    model = "llm-book/bert-base-japanese-v3-marc_ja"
)
positive_text = "世界には言葉がわからなくても感動する言葉がある"
print(text_classification_pipeline(positive_text)[0])

{'label': 'positive', 'score': 0.9990087151527405}


In [3]:
negative_text = "世界には言葉が出ないほどひどい言葉がある"
print(text_classification_pipeline(negative_text)[0])

{'label': 'negative', 'score': 0.9494393467903137}


In [4]:
nli_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-jnli")
text = "二人の男性がジェット機を見ています"
entailment_text = "ジェット機を見ている人が二人います"

print(nli_pipeline({"text": text, "text_pair": entailment_text}))

{'label': 'entailment', 'score': 0.9964311122894287}


NLI: Natural Language Interface
１つの文（前提文）に対して別の文（仮設文）が真(Entailment), 偽(Contradiction), 不明(Neutral)であるかを判定するものである。


In [5]:
contradiction_text = "2人の男性が飛んでいます"
print(nli_pipeline({"text": text, "text_pair": contradiction_text}))

{'label': 'contradiction', 'score': 0.9743013381958008}


In [6]:
neutral_text = "二人の男性が、白い飛行機を眺めています"
print(nli_pipeline({"text":text, "text_pair": neutral_text}))

{'label': 'neutral', 'score': 0.9579222202301025}


In [7]:
text_sim_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-jsts",
    function_to_apply = "none",
)
text = "川縁でサーフボードを持った人たちがいます"
sim_text = "私はサーフィンが好きだ"

result = text_sim_pipeline({"text": text, "text_pair": sim_text})
print(result["score"])

1.4963757991790771


In [8]:
dissim_text = "トイレの壁に黒いタオルがかけられています"
result = text_sim_pipeline({"text": text, "text_pair": dissim_text})
print(result["score"])

0.052118077874183655


In [9]:
from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(
    model = 'llm-book/bert-base-japanese-v3-unsup-simcse-jawiki',
    task = 'feature-extraction',
)

text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]

sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

0.7431767582893372


In [10]:
dissim_emb = sim_enc_pipeline(dissim_text, return_tensors=True)[0][0]

dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())

0.43792033195495605


In [11]:
from pprint import pprint

ner_pipeline = pipeline(
    model = 'llm-book/bert-base-japanese-v3-ner-wikipedia-dataset',
    aggregation_strategy='simple',
)

text='大谷翔平は岩手県水沢市出身のプロ野球選手'
pprint(ner_pipeline(text))

[{'end': None,
  'entity_group': '人名',
  'score': 0.99823624,
  'start': None,
  'word': '大谷 翔平'},
 {'end': None,
  'entity_group': '地名',
  'score': 0.9986874,
  'start': None,
  'word': '岩手 県 水沢 市'}]


NER -> Named Face Transformers　パイプラインを意味する
    ・複数のトークンが同じエンティティグループ（例：地名・人名）に属する場合にそれらのトークンをどのように集約するかを指定するためのオプションである。

In [12]:
text2text_pipeline = pipeline(
    model = 'llm-book/t5-base-long-livedoor-news-corpus'
)
article = 'ついに始まった三連休。テレビを見ながら過ごしている人も多いのではないだろうか？今夜おすすめなのはなんと言ってもNHKスペシャル世界を変えた男だな'
self_article = '僕の名前は古賀勇大、九州大学大学院に通っている修士1年生です。今は就職活動が忙しいので全然研究に着手できていません。研究は自然言語処理を用いて認知症に関連する研究を行っていきたいと考えていますが、データ集めに苦労していて何も手が進んでいません。周りの目が怖いです。'
print(text2text_pipeline(article)[0]['generated_text'])
print(text2text_pipeline(self_article)[0]['generated_text'])

今夜はNHKスペシャル世界を変えた男だな
認知症の研究をしています。周りの目が怖いです【インタビュー】


In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('abeja/gpt2-large-japanese')
tokenizer.tokenize('今日は天気が良いので')


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


['▁', '今日', 'は', '天気', 'が良い', 'の', 'で']

In [20]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    'abeja/gpt2-large-japanese'
)

inputs = tokenizer('今日は天気が良いので', return_tensors='pt')

outputs = model.generate(
    **inputs,
    max_length=30,
    pad_token_id=tokenizer.pad_token_id
)

generate_text = tokenizer.decode(
    outputs[0], slip_special_tokens=True
)

print(generate_text)

今日は天気が良いので</s>外でお弁当を食べました。 外で食べると 気持ちがいいですね。 今日は
